Nella cartella *files* trovi 8 file:
- 2 file di testo
- 2 file audio
- 4 immagini,

con varie estensioni.

**Step 1**

Inizia creando, in un notebook, uno script Python che iteri in ordine alfabetico sui file della cartella files e, a seconda del tipo (audio, documento, immagine), li sposti nella relativa sottocartella (qui sotto trovi un esempio). Se la sottocartella non esiste, il tuo script dovrà crearla automaticamente.

Durante il ciclo, lo script deve stampare le informazioni dei file: nome, tipo e dimensione in byte. Questo è l'output desiderato:

In [35]:
import fileinput
import csv
import os

class dataFile(object):
    def __init__(self, name, type, size): 
        self.name = name 
        self.type = type
        self.size = size

    def show(self):
        print(self.name, self.type, self.size)

class fileCSV(object):
    def __init__(self, path): 
        self.namefile = os.path.basename(path)
        self.path = path

    def updateListDataInCsv(self, item):
        with fileinput.input(files=(self.namefile), inplace=True, mode='r') as f:
            reader = csv.DictReader(f)
            print(",".join(reader.fieldnames))  # print back the headers
            for row in reader:
                if row["name"] == item.name and row["type"] == item.type:
                    row["size(B)"] = str(item.size)
                print(",".join([row["name"], row["type"], row["size(B)"]]))

    def appendDataItem(self, item):
        fields=[item.name, item.type, str(item.size)]
        # fields=['first','second','third']
        with open(self.path, 'a',  newline='') as f:
            writer = csv.writer(f)
            writer.writerow(fields)

    def createNewCSV(self):
        headerList = ['name', 'type', 'size(B)']
        # open CSV file and assign header
        with open(self.namefile, 'w', newline='') as file:
            dw = csv.DictWriter(file, delimiter=',', 
                                fieldnames=headerList)
            dw.writeheader()


item1 = dataFile("ciao", "test", 788)
item2 = dataFile("pluto", "image", 46)
item3 = dataFile("ciao", "testo", 68)
myCsvFile = fileCSV("files/recap-2.csv")
myCsvFile.createNewCSV()
myCsvFile.appendDataItem(item1)
myCsvFile.appendDataItem(item2) 
myCsvFile.appendDataItem(item3)       

In [36]:
#FILL ME
import json
import re
import os
import shutil
import ntpath
import filecmp

# dictionary that maps the file-extensions and the folder where the file needs to be stored to
dicOfExtFile = {"docs": ["txt", "odt"], 
                "audio": ["mp3"],
                "image": ["png", "jpg", "jpeg" ]
               }

class FileOrganizer(object):
    def __init__(self, mainfolder, csvname): 
        self.mainfolder = mainfolder
        self.csvFile = fileCSV(csvname)
        # create a new file only if it's not present already
        pwd = os.path.abspath(os.getcwd())
        pathToCSVFile = os.path.join(pwd, csvname)
        if os.path.exists(pathToCSVFile) == False:
            self.csvFile.createNewCSV()    
             
    # create the folder in the current path
    def createFolder(self, folderName):
        # get the current path 
        pwd = os.path.abspath(os.getcwd())
        pathToFolder = os.path.join(pwd, self.mainfolder, folderName)
        # create the folder (if it is not there yet)
        try:
            res = os.mkdir(pathToFolder) 
        except FileExistsError:
            res = ""

    def moveFileInFolder(self, folderName, nameFile):
        pwd = os.path.abspath(os.getcwd())
        pathToOriginFile = os.path.join(pwd, self.mainfolder, nameFile)
        pathToDestinationFile = os.path.join(pwd, "files", folderName, nameFile)
        # Check the following scenarios:
        # if the file already exists and is the same: remove the file and do nothing with csv
        # if the file already exists and is different (e.g. different size): overwrite the file
        #                                                                    and update csv
        # if the file does not exist: move it and update csv
        if os.path.exists(pathToDestinationFile):
            if filecmp.cmp(pathToDestinationFile, pathToOriginFile):
                os.remove(pathToDestinationFile) # ?necessary?
            else:
                shutil.move(pathToOriginFile, pathToDestinationFile)
                # in this case update the entry file in csv
                data = self.printHandleFile(pathToDestinationFile)
                self.csvFile.updateListDataInCsv(data)

        else:
            shutil.move(pathToOriginFile, pathToDestinationFile)
            data = self.printHandleFile(pathToDestinationFile)
            # in this case append the entry file in csv
            self.csvFile.appendDataItem(data)

    # get the folder name that needs to be created based on the extension file based on map dicOfExtFile
    def getKey(self, fileExt):
        test = fileExt
        key = [dictionary for dictionary, listOfExts in dicOfExtFile.items() if test in listOfExts]
        if key:
            # convert the extracted key in string and trimmer/clean the key (e.g. from ['audio'] to audio)
            keyString = json.dumps(key)
            keyString = re.sub(r'[^a-zA-Z0-9 ]',r'',keyString)
            return keyString
        else:
            #key not found
            return


    def loopFilesFolder(self):
        pwd = os.path.abspath(os.getcwd())
        pathToFiles =  os.path.join(pwd, self.mainfolder)
        for filename in os.listdir(pathToFiles):
            f = os.path.join(pathToFiles, filename)
            # checking if it is a file
            self.handleFile(f)  

    def handleFile(self, pathToNewFile):
        if os.path.isfile(pathToNewFile):
            filename = ntpath.basename(pathToNewFile)
            # get the extension file and remove the . (e.g. '.jpg')
            ext = os.path.splitext(filename)[1][1:]
            folderName = self.getKey(ext)
            # check folderName is not empty
            if folderName:
                self.createFolder(folderName)
                self.moveFileInFolder(folderName, filename)

    def printHandleFile(self, pathToNewFile):
        filename = ntpath.basename(pathToNewFile)
        name, ext = os.path.splitext(filename)
        ext = ext[1:]
        type = self.getKey(ext)
        file_stats = os.stat(pathToNewFile)
        print("{} type:{} size{}B".format(name,type, file_stats.st_size))
        d = dataFile(name, type, file_stats.st_size)
        return d


def getFileName(filename):
    return os.path.join("files", filename)

fileOrganizer = FileOrganizer("files", "files/recap.csv")
fileOrganizer.loopFilesFolder()



bw type:image size94926B
ciao type:docs size0B
daffodil type:image size24657B
eclipse type:image size64243B
pippo type:docs size8299B
song1 type:audio size1087849B
song2 type:audio size764176B
trump type:image size10195B



**TODO LIST**

**Create new CSV**
* Create new CSV if is not already present
* Create the list NAME TYPE SIZE
* save the list in the created CSV file

**Update CSV**
* open the CSV file and put the content in a list
* update the list with the new info
* re-save the list in the existing CSV file

Also before move a file, check if the file is the same. Otherwise no move and no CSV update


Oltre a stamparne le informazioni via via che li sposti, tieni traccia dei file creando un documento *recap.csv* con le stesse informazioni. Trovi un esempio in questa cartella.

La struttura finale della cartella files dovrà essere:

        - files            
            - audio
                - song1.mp3
                - song2.mp3
            - docs
                - ciao.txt
                - pippo.odt
            - images
                - bw.png
                - daffodil.jpg
                - eclipse.png
                - trump.jpeg    
            - recap.csv

Commenta il codice con i passaggi che fai. Questo vale anche per i prossimi Step.

**Attenzione**: lo script, ogni volta che viene lanciato per spostare nuovi file, deve *aggiornare* (e non sovrascrivere) le sottocartelle e il file di recap. Per controllare che tutto funzioni correttamente, puoi aggiungere altri file alla cartella files e fare un test; oppure, puoi dividere gli 8 file originali in due gruppi e lasciarne uno per il test.

**Consiglio**: puoi usare le librerie *os*, *shutil* e *csv*. 
                
---

**Step 2**

Inserisci lo script che hai creato in un piccolo eseguibile (chiamalo *addfile.py* e posizionalo in questa cartella, a fianco del notebook) dotato di *interfaccia a linea di comando* (CLI).

Lo scopo dell'eseguibile è spostare un *singolo* file (che si trova nella cartella files) nella sottocartella di competenza, aggiornando il recap.

L'interfaccia dell'eseguibile ha come unico argomento (obbligatorio) il nome del file da spostare (comprensivo di formato, es: 'trump.jpeg'). Nel caso in cui il file passato come argomento non esista, l'interfaccia deve comunicarlo all'utente.

**Consiglio**: oltre alle precedenti, puoi usare le librerie *sys* e *argparse*.

---

**Step 3**

Una immagine in scala di grigio ha un solo livello di colore, una RGB ne ha 3, una RGBA 4 (l'ultimo è detto canale *alpha*).

Il modulo *Image* della libreria *PIL* permette di caricare un'immagine, che può essere trasformata in un array NumPy attraverso la funzione *np.array*. A partire da tale array, è possibile capire se l'immagine caricata è in scala di grigio, RGB o RGBA.

Aggiungi al notebook dello Step 1 uno script che iteri sulla sottocartella *images* e costruisca una tabella riassuntiva come questa (prodotta con la libreria *tabulate*):

In [26]:
# Import PIL, tabulate and numpy modules
from PIL import Image
from tabulate import tabulate
import numpy as np
  
class ImageInfo(object):
    imageName = ""
    h = 0
    w = 0
    gs = 0
    r = 0
    g = 0
    b = 0
    a = 0
    imageType = ""

    def __init__(self, path): 
        im = Image.open(path)
        self.imageName = os.path.basename(path) 
        self.pix = np.array(im)

    def getAllinfo(self):
        self.getImageInfo()
        self.getAvarageBaseOnImage()
        return [self.imageName,self.h, self.w, self.gs, self.r, self.g, self.b, self.a]

    def getImageInfo(self):
        # image shape as (H, W, D)
        self.h = self.pix.shape[0]
        self.w = self.pix.shape[1]
        try:
            if self.pix.shape[2] == 3:
                computeRGB(self.pix)
                self.imageType = "rgb"
            elif self.pix.shape[2] == 4:
                computeRGBA(self.pix)
                self.imageType = "rgba"
        except IndexError:
            computeGreyscale(self.pix)
            self.imageType = "greyscale"

    def getAvarageBaseOnImage(self):
        if self.imageType == "greyscale":
            avarage = self.computeAvarage(sum = 0)
            self.gs = avarage
        if self.imageType == "rgb":
            avarage = self.computeAvarage(sum = [0,0,0])
            self.r = avarage[0]
            self.g = avarage[1]
            self.b = avarage[2]
        if self.imageType == "rgba":
            avarage = self.computeAvarage(sum = [0,0,0,0])
            self.r = avarage[0]
            self.g = avarage[1]
            self.b = avarage[2]
            self.a = avarage[3]       
            
    def computeAvarage(self, sum):
        for x in range(self.pix.shape[0]):
            for y in range(self.pix.shape[1]):
                sum += self.pix[x,y]
        avarage = sum / (self.pix.shape[0] * self.pix.shape[1])
        return avarage    

def computeGreyscale(im):
    sum = 0
    for x in range(im.shape[0]):
        for y in range(im.shape[1]):
            sum += im[x,y]
    gs = sum / (im.shape[0] * im.shape[1])

def computeRGB(im):
    sum = [0,0,0]
    for x in range(im.shape[0]):
        for y in range(im.shape[1]):
            sum += im[x,y]
    gs = sum / (im.shape[0] * im.shape[1])

def computeRGBA(im):
    sum = [0,0,0,0]
    for x in range(im.shape[0]):
        for y in range(im.shape[1]):
            sum += im[x,y]
    gs = sum / (im.shape[0] * im.shape[1])
    r = gs[0]
    g = gs[1]


def loopImageFolder():
        data = [["name", "height", "width","grayscale","R","G","B","ALPHA"]]
        table = tabulate(data, headers='firstrow',tablefmt='grid')

        pwd = os.path.abspath(os.getcwd())
        pathToImageFolder =  os.path.join(pwd, "files","image")
        for filename in os.listdir(pathToImageFolder):
            f = os.path.join(pathToImageFolder, filename)
            iminfo = ImageInfo(f)
            d = iminfo.getAllinfo()
            data.append(d)
        table = tabulate(data, headers='firstrow',tablefmt='grid')    
        print(table)             

loopImageFolder()


+--------------+----------+---------+-------------+----------+----------+----------+---------+
| name         |   height |   width |   grayscale |        R |        G |        B |   ALPHA |
+==============+==========+=========+=============+==========+==========+==========+=========+
| bw.png       |      512 |     512 |     21.4801 |   0      |   0      |  0       |   0     |
+--------------+----------+---------+-------------+----------+----------+----------+---------+
| daffodil.jpg |      500 |     335 |      0      | 109.251  |  85.5552 |  4.96729 |   0     |
+--------------+----------+---------+-------------+----------+----------+----------+---------+
| eclipse.png  |      256 |     256 |      0      | 109.05   | 109.522  | 39.849   | 133.591 |
+--------------+----------+---------+-------------+----------+----------+----------+---------+
| trump.jpeg   |      183 |     275 |      0      |  97.0108 |  98.9941 | 90.9238  |   0     |
+--------------+----------+---------+-------------

Oltre al nome del file, la tabella riporta:

- altezza dell'immagine, in pixel
- larghezza dell'immagine, in pixel
- se l'immagine è in scala di grigio, la colonna *grayscale* indica la media dei valori dell'unico livello di colore
- se l'immagine è a colori, le altre colonne indicano la media dei valori di ogni livello di colore.

---

**Dovrai consegnare**:
- un notebook con gli Step 1 e 3; per semplicità puoi chiamarlo come questo
- addfile.py con quanto richiesto dallo Step 2.